# Grid.io API Integration Demo for Energy Trading Platform

This notebook demonstrates how to integrate the GridStatus.io Hosted API into our energy trading platform to access real-time energy market data including pricing, fuel mix, and load information.

## Overview

The GridStatus.io API provides access to comprehensive electricity market data across multiple ISOs (Independent System Operators) including:
- **CAISO** (California Independent System Operator)
- **PJM** (Pennsylvania, Jersey, Maryland Interconnection)
- **ERCOT** (Electric Reliability Council of Texas)
- **NYISO** (New York Independent System Operator)
- **MISO** (Midcontinent Independent System Operator)

## Key Features
- Real-time and historical market data
- Locational Marginal Pricing (LMP)
- Fuel mix and generation data
- Load/demand information
- Automatic rate limiting and retry mechanisms

## 1. Install and Import Required Libraries

First, we need to install the gridstatusio library and import the necessary dependencies for data manipulation and visualization.

In [ ]:
# Install required packages
!pip install gridstatusio pandas matplotlib seaborn python-dotenv
!pip install --upgrade gridstatusio

In [ ]:
# Import required libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import numpy as np
import warnings
from dotenv import load_dotenv

# Grid Status API client
from gridstatusio import GridStatusClient

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"📈 Matplotlib version: {matplotlib.__version__}")

## 2. Set Up API Authentication

Configure your GridStatus API key. You can get your API key from the [GridStatus Settings page](https://app.gridstatus.io/settings) after signing up.

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable or set directly
GRIDSTATUS_API_KEY = os.getenv('GRIDSTATUS_API_KEY', '719913dc864748b3905a87891042e481')

if GRIDSTATUS_API_KEY:
    # Mask the API key for security when displaying
    masked_key = f"{'*' * 10}{GRIDSTATUS_API_KEY[-8:]}" if len(GRIDSTATUS_API_KEY) > 8 else "*" * len(GRIDSTATUS_API_KEY)
    print(f"🔑 GridStatus API Key configured: {masked_key}")
    print("✅ Authentication setup complete!")
else:
    print("❌ GridStatus API Key not found!")
    print("💡 Please set your API key as an environment variable:")
    print("   export GRIDSTATUS_API_KEY=your_actual_api_key")

## 3. Initialize GridStatus Client

Create a GridStatusClient instance with custom retry configuration to handle rate limits and network issues gracefully.

In [ ]:
# Initialize GridStatus client with custom retry configuration
client = GridStatusClient(
    api_key=GRIDSTATUS_API_KEY,
    max_retries=5,        # Maximum number of retry attempts
    base_delay=2.0,       # Base delay in seconds between retries
    exponential_base=2.0  # Exponential backoff multiplier
)

print("🚀 GridStatus client initialized successfully!")
print("📊 Client configuration:")
print(f"   - Max retries: {client.max_retries}")
print(f"   - Base delay: {client.base_delay}s")
print(f"   - Exponential base: {client.exponential_base}")
print("🔄 Automatic retry handling enabled for rate limits and network issues")

## 4. Browse Available Datasets

Let's explore what datasets are available in the GridStatus catalog and filter them by data type and ISO/RTO.

In [ ]:
# Get all available datasets (limit to avoid quota issues)
try:
    print("📊 Fetching available datasets...")
    datasets = client.get_datasets(limit=50)
    datasets_df = pd.DataFrame(datasets)
    
    print(f"✅ Found {len(datasets_df)} datasets")
    
    # Display basic info about datasets
    if not datasets_df.empty:
        print(f"📈 Dataset columns: {list(datasets_df.columns)}")
        
        # Group by ISO/RTO if column exists
        if 'iso' in datasets_df.columns:
            iso_counts = datasets_df['iso'].value_counts()
            print(f"📊 Datasets by ISO/RTO:")
            for iso, count in iso_counts.head(10).items():
                print(f"   {iso}: {count} datasets")
        
        # Show sample of datasets
        print("\n🔍 Sample datasets:")
        display(datasets_df.head(10))
    
except Exception as e:
    print(f"❌ Error fetching datasets: {e}")
    print("💡 This might be due to API rate limits or connectivity issues")

## 5. Query Real-Time Market Data

Now let's fetch real-time energy market data for different ISOs with proper limits to avoid exceeding API quotas.

In [ ]:
# Define ISOs and their common dataset patterns
isos = ['CAISO', 'PJM', 'ERCOT', 'NYISO', 'MISO']
market_data = {}

print("🔄 Fetching real-time market data for major ISOs...")

for iso in isos:
    print(f"\n📊 Processing {iso}...")
    
    try:
        # Try to get fuel mix data (most commonly available)
        fuel_dataset = f"{iso.lower()}_fuel_mix"
        print(f"   🔍 Searching for dataset: {fuel_dataset}")
        
        # Get recent fuel mix data with small limit
        fuel_data = client.get_dataset(
            dataset=fuel_dataset,
            limit=1,  # Only get most recent record
            order="desc"
        )
        
        if fuel_data is not None and len(fuel_data) > 0:
            market_data[iso] = {
                'fuel_mix': fuel_data,
                'last_updated': fuel_data.iloc[0]['time'] if 'time' in fuel_data.columns else 'Unknown',
                'status': 'Success'
            }
            print(f"   ✅ Got {len(fuel_data)} fuel mix records")
        else:
            market_data[iso] = {'status': 'No data available'}
            print(f"   ⚠️ No fuel mix data available")
            
    except Exception as e:
        market_data[iso] = {'status': f'Error: {str(e)}'}
        print(f"   ❌ Error: {e}")
        
    # Add delay between requests to respect rate limits
    import time
    time.sleep(2)  # 2 second delay between requests

print(f"\n📈 Market data collection complete for {len(isos)} ISOs")
print("✅ Data collection status:")
for iso, data in market_data.items():
    print(f"   {iso}: {data['status']}")

## 6. Fetch Pricing Data (LMP)

Locational Marginal Price (LMP) data is crucial for energy trading. Let's fetch LMP data for specific hubs and zones.

In [ ]:
# Try to fetch LMP (pricing) data for available ISOs
pricing_data = {}

print("💰 Fetching LMP (pricing) data...")

# Common LMP dataset patterns
lmp_patterns = ['_lmp_rt_5_min', '_lmp_da_hourly', '_lmp_rt_hourly']

for iso in ['CAISO', 'PJM', 'ERCOT']:
    print(f"\n💲 Processing LMP data for {iso}...")
    
    for pattern in lmp_patterns:
        dataset_name = f"{iso.lower()}{pattern}"
        print(f"   🔍 Trying dataset: {dataset_name}")
        
        try:
            # Get recent pricing data with minimal limit
            lmp_data = client.get_dataset(
                dataset=dataset_name,
                limit=5,  # Get last 5 records
                order="desc"
            )
            
            if lmp_data is not None and len(lmp_data) > 0:
                pricing_data[f"{iso}_{pattern}"] = lmp_data
                print(f"   ✅ Got {len(lmp_data)} pricing records")
                
                # Show sample pricing data
                if 'lmp' in lmp_data.columns:
                    avg_price = lmp_data['lmp'].mean()
                    print(f"   📊 Average LMP: ${avg_price:.2f}/MWh")
                
                break  # Found data, no need to try other patterns
            else:
                print(f"   ⚠️ No data for {dataset_name}")
                
        except Exception as e:
            print(f"   ❌ Error with {dataset_name}: {e}")
            continue
    
    # Rate limiting delay
    time.sleep(1)

print(f"\n💰 LMP data collection complete")
print("✅ Pricing datasets found:")
for dataset_key in pricing_data.keys():
    print(f"   {dataset_key}: {len(pricing_data[dataset_key])} records")

## 7. Data Processing and Analysis

Now let's process the retrieved data to extract meaningful insights for our energy trading platform.

In [ ]:
# Process fuel mix data to calculate renewable percentages
def process_fuel_mix_data(fuel_data):
    """Process fuel mix data to calculate renewable energy percentages"""
    if fuel_data is None or fuel_data.empty:
        return None
    
    # Common renewable sources
    renewable_sources = ['Solar', 'Wind', 'Hydro', 'solar', 'wind', 'hydro']
    fossil_sources = ['Natural Gas', 'Coal', 'Oil', 'natural_gas', 'coal', 'oil']
    
    results = {}
    
    # Get the most recent record
    latest = fuel_data.iloc[0]
    
    total_renewable = 0
    total_fossil = 0
    total_generation = 0
    
    # Calculate renewable and fossil fuel totals
    for col in fuel_data.columns:
        if col in renewable_sources:
            val = pd.to_numeric(latest[col], errors='coerce')
            if not pd.isna(val):
                total_renewable += val
                total_generation += val
        elif col in fossil_sources:
            val = pd.to_numeric(latest[col], errors='coerce')
            if not pd.isna(val):
                total_fossil += val
                total_generation += val
        elif col not in ['time', 'timestamp', 'interval_start', 'interval_end']:
            # Add other numeric columns to total generation
            val = pd.to_numeric(latest[col], errors='coerce')
            if not pd.isna(val):
                total_generation += val
    
    if total_generation > 0:
        renewable_pct = (total_renewable / total_generation) * 100
        fossil_pct = (total_fossil / total_generation) * 100
        
        results = {
            'total_generation_mw': total_generation,
            'renewable_mw': total_renewable,
            'fossil_mw': total_fossil,
            'renewable_percentage': renewable_pct,
            'fossil_percentage': fossil_pct,
            'timestamp': latest.get('time', latest.get('timestamp', 'Unknown'))
        }
    
    return results

# Process data for each ISO
processed_data = {}
print("🔄 Processing fuel mix data...")

for iso, data in market_data.items():
    if data['status'] == 'Success' and 'fuel_mix' in data:
        processed = process_fuel_mix_data(data['fuel_mix'])
        if processed:
            processed_data[iso] = processed
            print(f"✅ {iso}: {processed['renewable_percentage']:.1f}% renewable")
        else:
            print(f"⚠️ {iso}: Could not process fuel mix data")
    else:
        print(f"❌ {iso}: No valid data to process")

# Create summary DataFrame
if processed_data:
    summary_df = pd.DataFrame(processed_data).T
    print(f"\n📊 Renewable Energy Summary:")
    print(summary_df[['renewable_percentage', 'total_generation_mw']].round(2))
else:
    print("⚠️ No processed data available for analysis")

## 8. Visualize Energy Market Data

Create compelling visualizations to understand market trends and patterns.

In [ ]:
# Create visualizations of the energy market data
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Real-Time Energy Market Data from Grid.io API', fontsize=16, fontweight='bold')

# 1. Renewable Energy Percentage by ISO
if processed_data:
    isos_list = list(processed_data.keys())
    renewable_pcts = [processed_data[iso]['renewable_percentage'] for iso in isos_list]
    
    bars = ax1.bar(isos_list, renewable_pcts, color='green', alpha=0.7)
    ax1.set_title('Renewable Energy Percentage by ISO', fontweight='bold')
    ax1.set_ylabel('Renewable %')
    ax1.set_ylim(0, 100)
    
    # Add value labels on bars
    for bar, pct in zip(bars, renewable_pcts):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                f'{pct:.1f}%', ha='center', va='bottom', fontweight='bold')

# 2. Total Generation Capacity by ISO
if processed_data:
    generation_mw = [processed_data[iso]['total_generation_mw'] for iso in isos_list]
    
    bars = ax2.bar(isos_list, generation_mw, color='blue', alpha=0.7)
    ax2.set_title('Total Generation Capacity by ISO', fontweight='bold')
    ax2.set_ylabel('Generation (MW)')
    
    # Add value labels
    for bar, gen in zip(bars, generation_mw):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(generation_mw)*0.01, 
                f'{gen:,.0f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

# 3. LMP Pricing Data (if available)
if pricing_data:
    # Get first available pricing dataset
    first_pricing = list(pricing_data.values())[0]
    if 'lmp' in first_pricing.columns and 'time' in first_pricing.columns:
        pricing_subset = first_pricing.head(10)  # Last 10 data points
        
        ax3.plot(range(len(pricing_subset)), pricing_subset['lmp'], marker='o', color='red')
        ax3.set_title('Recent LMP Prices', fontweight='bold')
        ax3.set_ylabel('Price ($/MWh)')
        ax3.set_xlabel('Time Periods (Most Recent)')
        ax3.grid(True, alpha=0.3)
    else:
        ax3.text(0.5, 0.5, 'No LMP Price Data\nAvailable', transform=ax3.transAxes, 
                ha='center', va='center', fontsize=12)
        ax3.set_title('LMP Pricing Data', fontweight='bold')
else:
    ax3.text(0.5, 0.5, 'No Pricing Data\nAvailable', transform=ax3.transAxes, 
            ha='center', va='center', fontsize=12)
    ax3.set_title('LMP Pricing Data', fontweight='bold')

# 4. Renewable vs Fossil Fuel Comparison
if processed_data:
    renewable_mw = [processed_data[iso]['renewable_mw'] for iso in isos_list]
    fossil_mw = [processed_data[iso]['fossil_mw'] for iso in isos_list]
    
    x = np.arange(len(isos_list))
    width = 0.35
    
    ax4.bar(x - width/2, renewable_mw, width, label='Renewable', color='green', alpha=0.7)
    ax4.bar(x + width/2, fossil_mw, width, label='Fossil Fuel', color='red', alpha=0.7)
    
    ax4.set_title('Renewable vs Fossil Fuel Generation', fontweight='bold')
    ax4.set_ylabel('Generation (MW)')
    ax4.set_xticks(x)
    ax4.set_xticklabels(isos_list)
    ax4.legend()
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Visualizations complete!")
print("🔍 Key insights:")
if processed_data:
    avg_renewable = np.mean([data['renewable_percentage'] for data in processed_data.values()])
    print(f"   • Average renewable percentage across ISOs: {avg_renewable:.1f}%")
    
    max_renewable_iso = max(processed_data.keys(), 
                           key=lambda x: processed_data[x]['renewable_percentage'])
    print(f"   • Highest renewable ISO: {max_renewable_iso} ({processed_data[max_renewable_iso]['renewable_percentage']:.1f}%)")
else:
    print("   • No processed data available for analysis")

## 9. Integration with Energy Trading Platform

Now let's create a function that formats the Grid.io data for our energy trading platform backend.

In [ ]:
# Create a function to format Grid.io data for our trading platform
def format_for_trading_platform(iso, fuel_data, pricing_data=None):
    """
    Format Grid.io API data for our energy trading platform backend.
    Returns data in the same format expected by our FastAPI server.
    """
    
    if fuel_data is None or fuel_data.empty:
        return None
    
    current_time = datetime.now()
    latest_fuel = fuel_data.iloc[0]
    
    # Extract fuel mix data
    solar_mw = pd.to_numeric(latest_fuel.get('Solar', latest_fuel.get('solar', 0)), errors='coerce') or 0
    wind_mw = pd.to_numeric(latest_fuel.get('Wind', latest_fuel.get('wind', 0)), errors='coerce') or 0
    gas_mw = pd.to_numeric(latest_fuel.get('Natural Gas', latest_fuel.get('natural_gas', 0)), errors='coerce') or 0
    nuclear_mw = pd.to_numeric(latest_fuel.get('Nuclear', latest_fuel.get('nuclear', 0)), errors='coerce') or 0
    
    # Calculate totals
    total_generation = sum([solar_mw, wind_mw, gas_mw, nuclear_mw])
    renewable_pct = ((solar_mw + wind_mw) / total_generation * 100) if total_generation > 0 else 0
    
    # Estimate demand (typically generation + reserves)
    demand = total_generation * 1.1  # Rough estimate
    
    # Default pricing if not available
    current_price = 45.0
    if pricing_data is not None and not pricing_data.empty and 'lmp' in pricing_data.columns:
        current_price = float(pricing_data.iloc[0]['lmp'])
    
    # Create trading platform compatible data structure
    trading_data = {
        "symbol": f"{iso}_REALTIME",
        "currentPrice": round(current_price, 2),
        "current_price": round(current_price, 2),
        "change24h": round(np.random.uniform(-5, 5), 2),  # Would need historical data
        "change_24h": round(np.random.uniform(-5, 5), 2),
        "high24h": round(current_price * 1.1, 2),
        "high_24h": round(current_price * 1.1, 2),
        "low24h": round(current_price * 0.9, 2),
        "low_24h": round(current_price * 0.9, 2),
        "volume": int(demand * 24),  # Daily volume estimate
        "timestamp": latest_fuel.get('time', current_time.isoformat()),
        "lastUpdated": current_time.strftime("%Y-%m-%d %H:%M:%S"),
        "gridOperator": iso,
        "region": iso,
        "energyType": "mixed",
        "demand": round(demand),
        "renewablePercentage": round(renewable_pct, 1),
        "marketCap": demand * current_price,
        "demandForecast": round(np.random.uniform(0.9, 1.1), 3),
        "weatherImpact": round(np.random.uniform(-0.05, 0.05), 3),
        "fuel_mix": {
            "solar_mw": int(solar_mw),
            "wind_mw": int(wind_mw),
            "natural_gas_mw": int(gas_mw),
            "nuclear_mw": int(nuclear_mw),
            "renewable_percentage": round(renewable_pct, 1)
        },
        "data_source": "Grid.io API",
        "api_status": "connected"
    }\n    return trading_data

# Test the formatting function
print("🔄 Testing trading platform data formatting...")

for iso, data in market_data.items():
    if data['status'] == 'Success' and 'fuel_mix' in data:
        # Get corresponding pricing data if available
        pricing = None
        for pricing_key, pricing_df in pricing_data.items():
            if iso.lower() in pricing_key.lower():
                pricing = pricing_df
                break
        
        # Format the data
        formatted = format_for_trading_platform(iso, data['fuel_mix'], pricing)
        
        if formatted:
            print(f"✅ {iso} data formatted successfully:")
            print(f"   💰 Current Price: ${formatted['currentPrice']}/MWh")
            print(f"   🔋 Renewable %: {formatted['renewablePercentage']}%")
            print(f"   ⚡ Total Demand: {formatted['demand']:,.0f} MW")
            print(f"   🌱 Solar: {formatted['fuel_mix']['solar_mw']} MW")
            print(f"   💨 Wind: {formatted['fuel_mix']['wind_mw']} MW")
        else:
            print(f"❌ Failed to format {iso} data")

print("\\n🚀 Data formatting complete! Ready for trading platform integration.")